In [ ]:
# # File created for initial data cleaning and exploration

In [1]:
import pandas as pd
from prophet import Prophet
from astral import moon
import datetime as dt
import matplotlib.pyplot as plt
from datetime import timedelta
%matplotlib inline

In [28]:
# Import Houston crime data and moon files
houston_data_file = 'Houston_Crime_Data_Source_File.csv'
moon_data_file = 'moon_data_export.csv'

# Read the data into a pandas dataframe
houston_df= pd.read_csv(houston_data_file)
moon_df = pd.read_csv(moon_data_file)

C:\Users\Sheila\AppData\Local\Temp\ipykernel_13180\2344452984.py:6: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  houston_df= pd.read_csv(houston_data_file)


In [31]:
moon_df.head()

,Moon Phase,Date,Time (Universal Time)
0,Last Quarter,2023-01-15,02:10
1,New Moon,2023-01-21,20:53
2,First Quarter,2023-01-28,15:19
3,Full Moon,2023-02-05,18:28
4,Last Quarter,2023-02-13,16:01


In [39]:
renamed_moon_df = moon_df[['Date', 'Moon Phase']]

renamed_moon_df['Date'] = pd.to_datetime(renamed_moon_df['Date'], errors='coerce')
renamed_moon_df.head()

C:\Users\Sheila\AppData\Local\Temp\ipykernel_13180\4232584438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renamed_moon_df['Date'] = pd.to_datetime(renamed_moon_df['Date'], errors='coerce')


,Date,Moon Phase
0,2023-01-15,Last Quarter
1,2023-01-21,New Moon
2,2023-01-28,First Quarter
3,2023-02-05,Full Moon
4,2023-02-13,Last Quarter


In [40]:
houston_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185474 entries, 0 to 1185473
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   Unnamed: 0         1185474 non-null  int64 
 1   Date               1185474 non-null  object
 2   Hour               1185474 non-null  object
 3   Offense Type       1185474 non-null  object
 4   Beat               1185458 non-null  object
 5   Premise            1169553 non-null  object
 6   BlockRange         1185474 non-null  object
 7   StreetName         1184583 non-null  object
 8   Type               1185474 non-null  object
 9   Suffix             1173737 non-null  object
 10  # offenses         1042377 non-null  object
 11  CleanedStreetName  1173220 non-null  object
dtypes: int64(1), object(11)
memory usage: 108.5+ MB


In [50]:
# Reduce the DataFrame to necessary columns
cleaned_houston_crime_data_df = houston_df[['Date', 'Offense Type']]
cleaned_houston_crime_data_df.rename(columns={'Offense' : 'Offense Type'}, inplace=True) 

# Fill empty Incident Address fields with 'missing'
cleaned_houston_crime_data_df.fillna('missing', inplace=True)

# Change Occurance Date to datetime format
cleaned_houston_crime_data_df['Date'] = pd.to_datetime(pd.to_datetime(cleaned_houston_crime_data_df['Date'], errors='coerce'))

# Display DataFrame info and first 5 rows
cleaned_houston_crime_data_df.head()


C:\Users\Sheila\AppData\Local\Temp\ipykernel_13180\2498261585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_houston_crime_data_df.rename(columns={'Offense' : 'Offense Type'}, inplace=True)
C:\Users\Sheila\AppData\Local\Temp\ipykernel_13180\2498261585.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_houston_crime_data_df.fillna('missing', inplace=True)
C:\Users\Sheila\AppData\Local\Temp\ipykernel_13180\2498261585.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,Date,Offense Type
0,2016-02-13,Aggravated Assault
1,2016-02-16,Theft
2,2016-02-19,Aggravated Assault
3,2016-02-19,Theft
4,2016-02-20,Burglary


In [46]:
cleaned_houston_crime_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185474 entries, 0 to 1185473
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   Date          1185474 non-null  datetime64[ns]
 1   Offense Type  1185474 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 18.1+ MB


In [49]:
offense_categories_df = cleaned_houston_crime_data_df[cleaned_houston_crime_data_df['Offense Type'].unique()]
offense_categories_df

KeyError: "None of [Index(['Aggravated Assault', 'Theft', 'Burglary', 'Auto Theft', 'Robbery',\n       'Rape', 'Murder', '1', 'Burglary                 ',\n       'Robbery                  ', 'Theft                    ',\n       'AutoTheft                ', 'Aggravated Assault       ',\n       'Rape                     ', 'Murder                   ',\n       'False pretenses, swindle', 'From coin-operated machine or device',\n       'Intimidation', 'All other offenses', 'Simple assault',\n       'Drug, narcotic violations', 'Driving under the influence',\n       'Trespass of real property', 'Weapon law violations',\n       'Destruction, damage, vandalism', 'Stolen property offenses',\n       'Shoplifting', 'Theft of motor vehicle parts or accessory',\n       'Theft from motor vehicle', 'Identify theft', 'Credit card, ATM fraud',\n       'All other larceny', 'Impersonation', 'Burglary, Breaking and Entering',\n       'Murder, non-negligent', 'Theft from building', 'Motor vehicle theft',

In [11]:
# Sort the data by Occurrance Date
sorted_crime_data_df = cleaned_houston_crime_data_df.sort_values(by='Date', ascending=True)

# Reset Index
sorted_crime_data_df = sorted_crime_data_df.reset_index(drop=True)
sorted_crime_data_df.head(-1)

,Date,Offense
0,2010-01-01,Burglary
1,2010-01-01,Theft
2,2010-01-01,Theft
3,2010-01-01,Rape
4,2010-01-01,Theft
...,...,...
1185468,2018-12-31,All other larceny
1185469,2018-12-31,"Burglary, Breaking and Entering"
1185470,2018-12-31,"Destruction, damage, vandalism"
1185471,2018-12-31,Simple assault


In [ ]:
# Create a window for "near full moon" (e.g., ±1 day)
full_moon_periods = pd.concat([
    full_moon_df['Date'] - timedelta(days=1),
    full_moon_df['Date'],
    full_moon_df['Date'] + timedelta(days=1)
]).drop_duplicates().reset_index(drop=True)

# Create a window for "near full moon" (e.g., ±1 day)
quarter_moon_periods = pd.concat([
    full_moon_df['Date'] - timedelta(days=1),
    full_moon_df['Date'],
    full_moon_df['Date'] + timedelta(days=1)
]).drop_duplicates().reset_index(drop=True)

# Create a window for "near full moon" (e.g., ±1 day)
full_moon_periods = pd.concat([
    full_moon_df['Date'] - timedelta(days=1),
    full_moon_df['Date'],
    full_moon_df['Date'] + timedelta(days=1)
]).drop_duplicates().reset_index(drop=True)

In [12]:
# Convert date columns to datetime format
sorted_crime_data_df['Date'] = pd.to_datetime(sorted_crime_data_df['Date'], errors='coerce')
moon_df['Date'] = pd.to_datetime(moon_df['Date'], errors='coerce')

# Filter moon data for full moon events
full_moon_df = moon_df[moon_df['Moon Phase'] == 'Full Moon']

# Create a window for "near full moon" (e.g., ±1 day)
full_moon_periods = pd.concat([
    full_moon_df['Date'] - timedelta(days=1),
    full_moon_df['Date'],
    full_moon_df['Date'] + timedelta(days=1)
]).drop_duplicates().reset_index(drop=True)

# Add a column to indicate if the date is within the full moon periods
sorted_crime_data_df['Full Moon'] = sorted_crime_data_df['Date'].isin(full_moon_periods)

# Group data by full moon indicator and calculate summary statistics
houston_crime_sorted = sorted_crime_data_df.groupby('Full Moon')['Offense'].agg(['count', 'size'])
houston_crime_sorted.rename(columns={'count': 'Total Incidents', 'size': 'Crimes'}, inplace=True)

# Convert 'Crimes' column to numeric
houston_crime_sorted['Crimes'] = houston_crime_sorted['Crimes'].astype(int)

# Add a column for Crime Rate (%)
houston_crime_sorted['Crime Rate (%)'] = (houston_crime_sorted['Crimes'] / houston_crime_sorted['Total Incidents']) * 100

# Display the adjusted crime summary
print(houston_crime_sorted)


           Total Incidents   Crimes  Crime Rate (%)
Full Moon                                          
False              1117089  1117089           100.0
True                 68385    68385           100.0


In [13]:
# Create a dictionary of full moon and other counts for total rows (i.e. individual crimes)
moon_phase_crimes = houston_crime_sorted["Moon Phase"].value_counts().to_dict()

# Replace specific moon phases with "Other"
moon_phase_crimes["Other"] = moon_phase_crimes.pop("New Moon", 0) + moon_phase_crimes.pop("First Quarter", 0) + moon_phase_crimes.pop("Last Quarter", 0)
moon_phase_crimes 

KeyError: 'Moon Phase'

In [27]:
# Get a total count for non-full-moon dates (i.e. days with "Other")
other_dates = merged_Houston_df[merged_Houston_df["Moon Phase"] == "Other"]
days_other_moon = other_dates["Date"].nunique()
days_other_moon

3041

In [28]:
# Get a total count for individual crimes in each condition
days_full_moon = 124
full_moon_crimes = moon_phase_crimes.get("Full Moon")
other_moon_crimes = moon_phase_crimes.get("Other")
full_moon_crimes, other_moon_crimes 

(22846, 67716)

In [29]:
# Do the math for crime rate and percentage increase
full_crime_rate = full_moon_crimes / days_full_moon
other_crime_rate = other_moon_crimes / days_other_moon
percent_increase = (full_crime_rate / other_crime_rate - 1) * 100

print(f"The crime rate during a full moon is {full_crime_rate:.2f} against a control rate of {other_crime_rate:.2f}.")
print(f"These initial findings indicate a increased crime rate of {percent_increase:.2f}% above the base line.")

The crime rate during a full moon is 184.24 against a control rate of 22.27.
These initial findings indicate a increased crime rate of 727.40% above the base line.
